In [2]:
import pandas as pd
import numpy as np
import os
import json
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import math
import re

#load the index file that has all the filenames and file urls
df = pd.read_csv("/Volumes/USB THOMAS/THESIS NEW/DATA/2. PREPARE MASTERFILE AND FACTOR FILE/final_index_file.csv")

## Here I just take the pre-processed 10K files from Laughran and Mcdonald to create my text sample

In [2]:
df['To_delete'] = df['filing_url'].apply(lambda x: x.split('/')[8])

df['Lookup_Value'] = df['To_delete'].apply(lambda x: x[0:20])

In [3]:
df = df.drop('To_delete',axis=1)

In [4]:
df['Year'] = df['Y_Q'].apply(lambda x: x[0:4])

In [5]:
df['Quarter'] = df['Y_Q'].apply(lambda x: x[-1])

In [6]:
df['text'] = np.nan

In [7]:
df = df.drop(["access_number", 'accepted_date', 'filing_url'], 1)

dictio = {}

def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

for i in tqdm(range(len(df))):
    
    if i % 1000 == 0: 
        print('i = {}'.format(i))
        
    if df['report_url'][i] in dictio:
        math.isnan(df['text'][i])
        df = df.drop(i)
        
        continue
        
    dictio[df['report_url'][i]] = i
    
    os.chdir("D:\\THOMAS THESIS\\DATA\\10KQ PER YEAR\\" + str(df['Year'][i]) + '\\' + 'QTR' + str(df['Quarter'][i]))
    y = os.listdir()
    u = df['cik'][i]
    u2 = '_' + str(u) + '_'
    z = [s for s in y if df['Lookup_Value'][i] in s]
    
    if z == []:
        dictio[df['report_url'][i]] = 'NA'
        print('RED_FLAG', i)
    else:
    
        res = [j for j in z if str(u2) in j]
    
        res2 = str(res).strip("'][")
    
        x = "D:\\THOMAS THESIS\\DATA\\10KQ PER YEAR\\" + str(df['Year'][i]) + '\\' + 'QTR' + str(df['Quarter'][i]) + '\\' + str(res2)
    
        #with open(x) as f:
            #contents = f.readlines()
            
        with open(x, 'r') as f:
            text = f.read()
        
        text2 = remove_html_tags(text)
        
        df.loc[i,'text']= re.sub(r'[\ \n]{2,}', ' ', text2)

C:\Users\marti\AppData\Local\Temp/ipykernel_9092/1787556921.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(["access_number", 'accepted_date', 'filing_url'], 1)
  0%|          | 5/8691 [00:00<03:40, 39.31it/s]

i = 0


 12%|█▏        | 1003/8691 [01:00<06:18, 20.34it/s]

i = 1000


 23%|██▎       | 2006/8691 [01:52<04:16, 26.11it/s]

...

 81%|████████  | 7003/8691 [05:13<00:57, 29.13it/s]

i = 7000


 92%|█████████▏| 8005/8691 [05:43<00:18, 37.50it/s]

i = 8000


100%|██████████| 8691/8691 [06:06<00:00, 23.69it/s]


In [13]:
df = df.drop(["Lookup_Value", 'Year', 'Quarter'], 1)

df.to_pickle("D:\THOMAS THESIS\DATA\MASTER INDEX FILES WITH TEXT DATA\IDX FILE WITH TEXT DATA.pkl")  

df.to_csv("D:\THOMAS THESIS\DATA\MASTER INDEX FILES WITH TEXT DATA\IDX FILE WITH TEXT DATA.csv")

C:\Users\marti\AppData\Local\Temp/ipykernel_9092/1655141114.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(["Lookup_Value", 'Year', 'Quarter'], 1)


In [3]:
unpickled_df = pd.read_pickle("D:\THOMAS THESIS\DATA\MASTER INDEX FILES WITH TEXT DATA\IDX FILE WITH TEXT DATA.pkl")  

# SCRAPE 8K FILES

In [88]:


df = df.drop(["access_number", 'accepted_date', 'filing_url'], 1)

dictio = {}

error_listje = []

for i in tqdm(range(len(df))):
    
    if i % 1000 == 0: 
        print('i = {}'.format(i), 'url = {}'.format(df['report_url'][i]))
    
    if df['report_url'][i] in dictio:
        math.isnan(df['text'][i])
        df = df.drop(i)
        
        continue
    
    path = str(df['report_url'][i])
    r = requests.get(path, headers={"User-Agent": "b2g"})
    soup = BeautifulSoup(r.text, "html.parser")
    
    soup2 = soup.get_text()
    
    u = 0
    
    for t in soup.find_all(['TABLE','td','tr','th']):
        t.decompose()
        u += 1
    
    if u < 5:
        error_listje.append(df['report_url'][i])
    
    divs = soup.find_all('document')
    
    textfile = ''
    
    dictio[df['report_url'][i]] = i
    
    for h in divs:
        textfile += h.get_text()
    
    df.loc[i,'text']= re.sub(r'[\ \n]{2,}', ' ', textfile)

C:\Users\marti\AppData\Local\Temp/ipykernel_3544/1590948597.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(["access_number", 'accepted_date', 'filing_url'], 1)
  0%|          | 1/8691 [00:00<22:34,  6.41it/s]

i = 0 url = https://www.sec.gov/Archives/edgar/data/106535/0000106535-96-000008.txt


 12%|█▏        | 1002/8691 [02:45<22:04,  5.80it/s] 

i = 1000 url = https://www.sec.gov/Archives/edgar/data/20290/000089965201500075/form10q3rdqtr2001.htm


 23%|██▎       | 2000/8691 [19:47<1:08:02,  1.64it/s] 

...

 81%|████████  | 7000/8691 [4:32:32<3:20:21,  7.11s/it] 

i = 7000 url = https://www.sec.gov/Archives/edgar/data/1130310/000113031017000017/cnp_10qx9302017.htm


 92%|█████████▏| 8000/8691 [5:31:11<05:29,  2.10it/s]   

i = 8000 url = https://www.sec.gov/ix?doc=/Archives/edgar/data/72903/000007290320000011/xcel1231201910-k.htm


100%|██████████| 8691/8691 [5:39:08<00:00,  2.34s/it]
